# Calculating thermal time indices and merging UAV data.

**Name**: Byron Evers<br/>
**Semester**: Spring 2019 <br/>
**Project area**: Agronomy

## Motivation
- Poland labs current UAV pipeline includes stitching photos and extracting plot level reflectance data through Agisoft software.
- Data received from this process is in either an Excel or csv file. 
- The data set includes reflectance values for 5 individual bands (R,G,B,RE and NIR) and 3 vegetative indices (NDVI, NDRE and GNDVI). 
- Currently a wide range of time scales are used for UAV data analysis. 
    * Days after planting
    * Day of Year
    * Thermal Time

<a name="BJE_objective"></a>
## Objective

1. Write a python function to calculate three thermal time indices for all UAV collection dates
    * growing degree days (GDD)
    * physiological days (Pdays)
    * biometeorological time (BMT) 
2. Merge all of the UAV reflectance data, plot level phenotypic data and the calculated thermal values into one .csv file.

**I would like to emphasize that the goal of this project is to create a single output file with UAV data and thermal time indices.**

<a name="BJE_step1"></a>
# Step 1: Import needed modules
To run ephm you may need to install it first. To do this please run **pip install ephem** command line or execute the cell bellow.

In [3]:
!pip install ephem

In [4]:
import pandas as pd
import numpy as np
import datetime
import ephem
import matplotlib.pyplot as plt
%matplotlib inline

<a name="BJE_step2"></a>
# Step 2: Download Data from the KSU Mesonet. 
* daily
* root = 'http://mesonet.k-state.edu/rest/stationdata/?'
* stn = 'Manhattan'
* start_time = "20140101000000" 
* end_time =  datetime.datetime.now().strftime('%Y%m%d%H%M%S') 

# Step 3: Format Data Frame 
* Date- class 'pandas._libs.tslibs.timestamps.Timestamp'
* Station- class 'str'
* Min daily temperature- class 'numpy.float64'
* Max daily temperature- class 'numpy.float64'
     

# Step 4: Define User Inputs
* The inputs needed are;
    * plantDate- YYYY-mm-dd
    * harvestDate- YYYY-mm-dd
    * lat- decimal degree latitude 
    * long- decimal degree longitude 
    * tbase- base temperature for the crop in degrees C, 5<sup>o</sup>C for this example
    * topt- optimumn growing temperature for the crop in degrees C. 17<sup>o</sup>C for this example
    * tmax- maximum growing temperature for the crop in degrees C. 30<sup>o</sup>C for this example
    * k- scale factor for the Pdays equation. This maybe be edited for other crops. 
    
**All of these parameters are designed so that users can adjust them to fit the needs of the individual project**

# Step 5: Calculate Photoperiod

To complete this the ephem module is used. The output is displayed as decimal hours with the times of sunrise and sunset on a 24 hour clock. Date frame must have the following columns;
* Date
* lat
* long

In [5]:
obs = ephem.Observer()
obs.long = ephem.degrees(long)
obs.lat = ephem.degrees(lat)
date =  df['Date'].dt.strftime('%Y/%m/%d')
sun = ephem.Sun(obs)

rise_data=[]
set_data=[]
photoperiod_data=[]

for i in date:
    obs.date = ephem.Date(i)
    r1 = obs.next_rising(sun)
    s1 = obs.next_setting(sun)
    r1_lt = ephem.Date(r1 - 5 * ephem.hour) #local time 
    (y, mn, d, h, min, s) = r1_lt.tuple()
    rise = ( h + min/60. + s/3600. )
    s1_lt = ephem.Date(s1 - 5 * ephem.hour) #local time
    (y, mn, d, h, min, s) = s1_lt.tuple()
    sunset = ( h + min/60. + s/3600. )
    photoperiod=sunset-rise
    rise_data.append(rise)
    set_data.append(sunset)
    photoperiod_data.append(photoperiod)
    df['Sunrise']= pd.DataFrame(rise_data)
    df['Sunset']= pd.DataFrame(set_data)
    df['PhotoPeriod']= pd.DataFrame(photoperiod_data)   
df.head(5)    

NameError: name 'long' is not defined

<img src="Fig/photoperiod.png" alt="sketch_image" width="600"/>

# Step 6: Define Growing Season

In [ ]:
def Growing_season (plant, harvest, df):
    """
    Sets the growing season form plant to harvest for each growing season 
   
     Input: Requires three inputs;
    1. harvest date string formatted as 'YYYY-MM-DD' this can be in the form of a stored variable or a direct string input
    2. Harvest date, string, formatted as 'YYYY-MM-DD' this can be in the form of a stored variable or a direct string input
    3. Pandas data frame with required date column formatted as 'YYYY-MM-DD'
    
    Output: Pandas data frame. With the dates of the growing season of interest. Recommend to save the ouput as a variable
        
    Byron Evers
    
    2019-03-06 
    """
    plant = np.datetime64(plant)
    harvest = np.datetime64(harvest)
    min_date = df.Date.min()
    min_date = min_date.to_datetime64()
    max_date = df.Date.max()
    max_date = max_date.to_datetime64()
    if min_date > plant:
        print('The planting date ' +str(plant)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
        return('The harvest date ' +str(plant)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
    if min_date > harvest:
        print('The harvest date ' +str(harvest)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
        return('The harvest date ' +str(harvest)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
    if max_date < harvest:
        print('The harvest date ' +str(harvest)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
        return('The harvest date ' +str(harvest)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
    if max_date < plant:
        print('The planting date ' +str(plant)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
        return('The planting date ' +str(plant)+' is beyond the range of the Mesonet data. The mesonet data goes from ' + str(df.Date.min())+ " to " + str(df.Date.max()))
    df = df.drop(df[df.Date < plant].index)
    df = df.drop(df[df.Date > harvest].index)
    return df

In [ ]:
#Call the function and store the data as a new data frame. In this example two data frames are created,df17 and df18, for the 2017 and 2018 winter wheat growing season
df17 = Growing_season('1916-10-17','2017-06-21',df)
df18 = Growing_season(plantDate,harvestDate,df)
# For df17 date as a string was used where as df18 used the stored variable from input cell. 
#This is just to demonstrate that either format is accepted with this function.

The planting date 1916-10-17 is beyond the range of the Mesonet data. The mesonet data goes from 2014-01-02 00:00:00 to 2019-04-14 00:00:00

# Step 7: Calculate GDD
* Tmax= daily maximum temperature
* Tmin= daily minimum temperature
* Tbase= base growing temperature for given crop
* Date = date of collection
$$GDD = \sum_{Planting}^{Harvest}(\frac{Tmax+Tmin}{2})-Tbase$$

# Step 8: Calculate Pdays 
* Tmax= daily maximum temperature
* Tmin= daily minimum temperature
* Tbase= base growing temperature for given crop
* topt= optimum growing temperature for given crop
* tmax= maximum growing temperature for given crop
* Date = date of collection
* k= scale factor
$$Pdays = \frac{1}{24}(5*P(T_1)+8*P(T_2)+8*P(T_3)+3*P(T_4))$$
**Where**
* $T_1=Tmin$
* $T_2=\frac{(2*Tmin)+Tmax}{3}$
* $T_3=\frac{Tmin+(2*Tmax)}{3}$
* $T_4=Tmax$

**And P is**
* $P=0$   When $T <=Tmin$
* $P=k*(1-\frac{(T-Topt)2}{(Topt-Tmin)2})$ when $Tmin <= T <=Topt$
* $P=k*(1-\frac{(T-Topt)2}{(Tmax-Topt)2})$ when $Topt <= T <=Tmax$
* $P=0$ when $T >=Tmax$

**And k is**
* $P=0$   When $T <=Tmin$

# Step 9: Calculate BMT
* par = dictonary of coefficents for each stage of development
* Tmax= daily maximum temperature
* Tmin= daily minimum temperature
* PhotoPeriod=daily photo period
* Date = date of collection
        

<a name="BJE_BMT"></a>
### Biometeorological Time
$$BMT = \sum_{Planting}^{Harvest}[a_1(L-a_0) + a_2(L-a_0)^2* {[b_1(Tmax-b_0) + b_2(Tmax-b_0)^2}+{d_1(Tmin-b_0) + d_2(Tmin-b_0)^2]}$$
or for simplicity:
$$BMT = \sum_{Planting}^{Harvest}V_1 {(V_2}+{V_3)}$$
**Where**
* $L$= daily photoperiod
* $a_0$= base daylength
* $b_0$= base temperature
* $a_1, a_2, b_1, b_2, d_1, d_2$ are response coefficients
    * For this project I used the coefficents detailed by Robertson (1968)

In [6]:
par = {'a0':0,'a1':0,'a2':0,'b0':0,'b1':0,'b2':0,'d1':0,'d2':0,
       'a0_PE':8.413,'a1_PE':1.005,'a2_PE':0,'b0_PE':0,'b1_PE':0.003512,'b2_PE':0.00005026,'d1_PE':0.000366,'d2_PE':0.000004282,
       'a0_EJ':9.413,'a1_EJ':1.005,'a2_EJ':0,'b0_EJ':0,'b1_EJ':0.003512,'b2_EJ':0.00005026,'d1_EJ':0.000366,'d2_EJ':0.000004282,
       'a0_JH':10.93,'a1_JH':0.9256,'a2_JH':0.06025,'b0_JH':0,'b1_JH':0.0002958,'b2_JH':0,'d1_JH':0.0003943,'d2_JH':0,
       'a0_HS':10.94,'a1_HS':1.389,'a2_HS':1,'b0_HS':0,'b1_HS':0.004458,'b2_HS':0,'d1_HS':0.00003109,'d2_HS':0,
       'a0_SR':14.38,'a1_SR':1.614,'a2_SR':1,'b0_SR':0,'b1_SR':0.007733,'b2_SR':0,'d1_SR':0.0003442,'d2_SR':0,}

In [10]:
def BMT (par,df):
    """
    Calculates an individual and cumulative value for BMT for each day in the data frame
    
    Input: Pandas data frame. Minimum required columns include:
        par = dictonary of coefficents for each stage of development
        Tmax= daily maximum temperature
        Tmin= daily minimum temperature
        PhotoPeriod=daily photo period
        Date = date of collection
        
        
    Output: Pandas data frame. In addition, the columns in the data frame input two additional columns will be calculated and added:
        BMT= The BMT value for that individual day
        cum_BMT = The cumulative BMT value for all days in the designated growing period.
    
    Byron Evers
    
    2019-05-05
    """
    BMT=[0]*len(df)
    cum_BMT=[0]*len(df)
    for i in range(1,len(df)):
        if cum_BMT[i-1] < 1:
            par['a0']=par['a0_PE']
            par['a1']=par['a1_PE']
            par['a2']=par['a2_PE']
            par['b0']=par['b0_PE']
            par['b1']=par['b1_PE']
            par['b2']=par['b2_PE']
            par['d1']=par['d1_PE']
            par['d2']=par['d2_PE']
        elif cum_BMT[i-1] >= 1 and cum_BMT[i-1]< 2:
            par['a0']=par['a0_EJ']
            par['a1']=par['a1_EJ']
            par['a2']=par['a2_EJ']
            par['b0']=par['b0_EJ']
            par['b1']=par['b1_EJ']
            par['b2']=par['b2_EJ']
            par['d1']=par['d1_EJ']
            par['d2']=par['d2_EJ']
        elif cum_BMT[i-1] >= 2 and cum_BMT[i-1]< 2.5:
            par['a0']=par['a0_JH']
            par['a1']=par['a1_JH']
            par['a2']=par['a2_JH']
            par['b0']=par['b0_JH']
            par['b1']=par['b1_JH']
            par['b2']=par['b2_JH']
            par['d1']=par['d1_JH']
            par['d2']=par['d2_JH']
        elif cum_BMT[i-1] >= 2.5 and cum_BMT[i-1]< 3:
            par['a0']=par['a0_HS']
            par['a1']=par['a1_HS']
            par['a2']=par['a2_HS']
            par['b0']=par['b0_HS']
            par['b1']=par['b1_HS']
            par['b2']=par['b2_HS']
            par['d1']=par['d1_HS']
            par['d2']=par['d2_HS']  
        else: 
            par['a0']=par['a0_SR']
            par['a1']=par['a1_SR']
            par['a2']=par['a2_SR']
            par['b0']=par['b0_SR']
            par['b1']=par['b1_SR']
            par['b2']=par['b2_SR']
            par['d1']=par['d1_SR']
            par['d2']=par['d2_SR']
        
        photo =df.PhotoPeriod
        photo= np.where(photo < 0,0,photo)
        tmax =df.Tmax
        tmax= np.where(tmax < 0,0,tmax)
        tmin =df.Tmin
        tmin= np.where(tmin < 0,0,tmin)
        if cum_BMT[i-1] < 0.25:
            v1=1
        else:
            v1=abs((par['a1']*(photo[i]-par['a0'])+(par['a2']*((photo[i]-par['a0'])**2))))
        v2=(par['b1']*(tmin[i]-par['b0'])+(par['b2']*((tmin[i]-par['b0'])**2)))
        v3=(par['d1']*(tmin[i]-par['b0'])+(par['d2']*((tmin[i]-par['b0'])**2)))
        BMT[i]=(v1*(v2+v3))

        T=cum_BMT[i-1]+BMT[i]
        cum_BMT[i]=T
    df['BMT']=BMT  
    df['cum_BMT']=cum_BMT   
    return df

# Step 10: Graph Results
Graphing the data allows for visual comparison. However, as stated in Step 9 the scale of the three thermal time indices are not equivalent. Therefore, when comparing the three indices a scale factor was needed. For this project the scale created was defined as;

$$scale = \frac{cumGDD.max()}{cumBMT.max()}$$

<img src="Fig/graph.png" alt="sketch_image" width="800"/>

# Step 11: Merge UAV and Thermal Time Index
* Merge UAV data and thermal time indices by "Date"

<img src="Fig/finaldf.png" alt="sketch_image" width="800"/>

## Conclusions
* Functions for all three indices are operational
* Code with in the functions needs to be optimized
* I would like to explore other thermal indices and collect additional field data to choose the best index to model time series reflectance data